In [1]:
import os
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import ccxt
import time
from dotenv import load_dotenv
from alpha_vantage.techindicators import TechIndicators


Bad key "text.kerning_factor" on line 4 in
C:\Users\Josh Bond\Anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
load_dotenv()

True

In [3]:
kraken_pub_key = os.getenv("KRAKEN_PUBLIC_KEY")
kraken_sec_key = os.getenv("KRAKEN_SECRET_KEY")
kraken = ccxt.kraken({"apiKey": kraken_pub_key, "secret": kraken_sec_key})

av_key = os.getenv("ALPHAVANTAGE_API_KEY")

In [5]:
# Getting BTC data and setting columns and date
btc_historical = kraken.fetch_ohlcv("BTC/USD", "1d")
btc_historical_df = pd.DataFrame(
    btc_historical, 
    columns=['date','open','high','low','close','volume']
)
btc_historical_df['date'] = pd.to_datetime(btc_historical_df['date'], unit='ms')
btc_historical_df = btc_historical_df.sort_values(by='date')

In [7]:
ti = TechIndicators(output_format = 'pandas')
btc_ema = ti.get_ema('BTCUSD', interval='daily', time_period=20)

In [8]:
btc_ema

(                   EMA
 date                  
 2011-09-19      8.3675
 2011-09-20      8.0373
 2011-09-21      7.8109
 2011-09-22      7.6060
 2011-09-23      7.4264
 2011-09-24      7.2639
 2011-09-25      7.1130
 2011-09-26      7.0118
 2011-09-27      6.8011
 2011-09-28      6.6220
 2011-09-29      6.4504
 2011-09-30      6.2951
 2011-10-04      6.1594
 2011-10-05      6.0471
 2011-10-06      5.9397
 2011-10-07      5.8312
 2011-10-08      5.7149
 2011-10-09      5.5506
 2011-10-10      5.3924
 2011-10-11      5.3084
 2011-10-12      5.1933
 2011-10-13      5.0730
 2011-10-14      4.9680
 2011-10-15      4.8805
 2011-10-18      4.7890
 2011-10-21      4.5463
 2011-10-22      4.3295
 2011-10-25      4.1953
 2011-10-26      4.0548
 2011-10-27      3.9248
 ...                ...
 2021-03-16  53570.3052
 2021-03-17  54079.1237
 2021-03-18  54418.9441
 2021-03-19  54763.6692
 2021-03-20  55081.8086
 2021-03-21  55298.2916
 2021-03-22  55183.3691
 2021-03-23  55104.6101
 2021-03-24  548

In [ ]:
ti = TechIndicators(key = av_key, output_format = 'pandas')
data, meta_data = ti.get_ema(symbol='CryptoCurrency:BTC', interval='daily', time_period=20)

In [ ]:
cc = CryptoCurrencies(key=av_key,output_format='pandas')
data, meta_data = cc.get_digital_currency_daily(symbol='BTC',market='USD')

In [ ]:
split1 = int(.65*len(btc_historical_df))
split2 = int(.5*len(val_test))


trainset = btc_historical_df[:split1]
val_test = btc_historical_df[split1:]
validation_set = val_test[:split2]
testset = val_test[split2:]

In [ ]:
av_key